In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta

In [3]:
# Fetch historical stock data
def fetch_stock_data(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    df = stock.history(start=start_date, end=end_date)
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

In [5]:
# Build and train an LSTM model for stock price prediction
def create_lstm_model(data, time_steps, future_days):
    # Normalize the data to a range between 0 and 1
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    # Prepare the data for LSTM with the given time steps
    X, y = [], []
    for i in range(time_steps, len(scaled_data) - future_days + 1):
        X.append(scaled_data[i - time_steps:i])
        y.append(scaled_data[i:i + future_days, 3])  # 3 corresponds to 'Close' column

    # Convert to numpy arrays for model training
    X, y = np.array(X), np.array(y)
    n_features = X.shape[2]

    # Define the LSTM model structure
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(time_steps, n_features)),
        LSTM(50, return_sequences=False),
        Dense(future_days)
    ])
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X, y, epochs=50, batch_size=32, validation_split=0.1, verbose=1)
    return model, scaler

In [7]:
# Predict future prices using the trained LSTM model
def predict_future_prices(model, scaler, last_sequence, future_days):
    last_sequence_scaled = scaler.transform(last_sequence)  # Scale the last sequence of input data
    predicted_scaled = model.predict(np.array([last_sequence_scaled]))  # Predict future prices

    # Reverse the scaling to get actual price values
    predicted = scaler.inverse_transform(np.column_stack((predicted_scaled[0], np.zeros((future_days, 4))))) 
    return predicted[:, 0]  # Return only the 'Close' price predictions

In [9]:
# Prepare data for the selected stock and generate future price predictions
def prepare_stock_data(symbol):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*5)  # Use 5 years of historical data
    data = fetch_stock_data(symbol, start_date, end_date)
    time_steps = 60  # Days of past data used to predict the future
    future_days = 15  # Predict prices for the next 15 days

    # Train the model and get the scaler
    model, scaler = create_lstm_model(data, time_steps, future_days)

    # Prepare the last 'time_steps' days of data for prediction
    last_sequence = data.iloc[-time_steps:].values
    future_prices = predict_future_prices(model, scaler, last_sequence, future_days)

    # Generate future dates and create a DataFrame with predictions
    future_dates = pd.date_range(start=end_date, periods=future_days)
    future_df = pd.DataFrame({'Date': future_dates, 'Predicted_Close': future_prices})
    
    return data, future_df


In [11]:
# Main execution
if __name__ == "__main__":
    symbol = "AAPL"  # Example stock symbol
    historical_data, future_data = prepare_stock_data(symbol)  # Fetch and predict

    print(f"Historical data shape: {historical_data.shape}")
    print(f"Future data shape: {future_data.shape}")
    print(f"Future predicted prices:\n{future_data}")

D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.1218 - val_loss: 0.0376
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0036 - val_loss: 0.0160
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0025 - val_loss: 0.0115
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0026 - val_loss: 0.0172
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0023 - val_loss: 0.0118
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0023 - val_loss: 0.0108
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0023 - val_loss: 0.0137
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0024 - val_loss: 0.0161
Epoch 9/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0023 - val_loss: 0.0103
Epoch 10/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0022 - val_loss: 0.0072
Epoch 11/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0023 - val_loss: 0.0055
Epoch 12/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0

D:\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
Historical data shape: (1258, 5)
Future data shape: (15, 2)
Future predicted prices:
                         Date  Predicted_Close
0  2024-11-16 04:25:10.636446       220.232273
1  2024-11-17 04:25:10.636446       221.493210
2  2024-11-18 04:25:10.636446       218.939624
3  2024-11-19 04:25:10.636446       218.296850
4  2024-11-20 04:25:10.636446       218.185498
5  2024-11-21 04:25:10.636446       216.796238
6  2024-11-22 04:25:10.636446       214.430621
7  2024-11-23 04:25:10.636446       216.574415
8  2024-11-24 04:25:10.636446       213.976688
9  2024-11-25 04:25:10.636446       213.733849
10 2024-11-26 04:25:10.636446       215.226552
11 2024-11-27 04:25:10.636446       208.972818
12 2024-11-28 04:25:10.636446       210.852468
13 2024-11-29 04:25:10.636446       206.950200
14 2024-11-30 04:25:10.636446       208.688680
